In [16]:
from swebench.harness.utils import load_swebench_dataset
from swebench.harness.test_spec import make_test_spec
instances = load_swebench_dataset(split="test", name="princeton-nlp/SWE-bench_Verified")

In [17]:
import swebench
swebench.__file__

'/Users/shrey/.local/share/virtualenvs/python-PN9waRBr/lib/python3.10/site-packages/swebench/__init__.py'

In [18]:
def get_instance(pattern=None, version=None):
    instances_list = []
    for instance in instances:
        if pattern and version:
            if pattern in instance["instance_id"] and instance["version"] == version:
                instances_list.append(instance)
        elif pattern and not version:
            if pattern in instance["instance_id"]:
                instances_list.append(instance)
        elif not pattern and version:
            if instance["version"] == version:
                instances_list.append(instance)
        else:
            instances_list.append(instance)
    return sorted(instances_list, key=lambda x: x["instance_id"])

In [22]:
insts = get_instance(pattern="7440")
inst = insts[0]
test_spec = make_test_spec(inst)


In [23]:
print(test_spec.eval_script)

#!/bin/bash
set -uxo pipefail
source /opt/miniconda3/bin/activate
conda activate testbed
cd /testbed
git config --global --add safe.directory /testbed
cd /testbed
git status
git show
git diff 9bb204dcabe6ba0fc422bf4a45ad0c79c680d90b
source /opt/miniconda3/bin/activate
conda activate testbed
python -m pip install -e .[test]
git checkout 9bb204dcabe6ba0fc422bf4a45ad0c79c680d90b tests/test_domain_std.py
git apply -v - <<'EOF_114329324912'
diff --git a/tests/test_domain_std.py b/tests/test_domain_std.py
--- a/tests/test_domain_std.py
+++ b/tests/test_domain_std.py
@@ -99,7 +99,7 @@ def test_glossary(app):
     text = (".. glossary::\n"
             "\n"
             "   term1\n"
-            "   term2\n"
+            "   TERM2\n"
             "       description\n"
             "\n"
             "   term3 : classifier\n"
@@ -114,7 +114,7 @@ def test_glossary(app):
     assert_node(doctree, (
         [glossary, definition_list, ([definition_list_item, ([term, ("term1",
                    

In [24]:
print(test_spec.env_dockerfile)
# print(test_spec.__dict__)

FROM --platform=linux/arm64/v8 sweb.base.arm64:latest

COPY ./setup_env.sh /root/
RUN chmod +x /root/setup_env.sh
RUN /bin/bash -c "source ~/.bashrc && /root/setup_env.sh"

WORKDIR /testbed/

# Automatically activate the testbed environment
RUN echo "source /opt/miniconda3/etc/profile.d/conda.sh && conda activate testbed" > /root/.bashrc



In [26]:
print(test_spec.setup_env_script)

#!/bin/bash
set -euxo pipefail
source /opt/miniconda3/bin/activate
conda create -n testbed python=3.9  -y
conda activate testbed
python -m pip install tox==4.16.0 tox-current-env==0.0.11



In [27]:
print(test_spec.instance_dockerfile)

FROM --platform=linux/arm64/v8 sweb.env.arm64.c6d251a05e0af7688b64fd:latest

COPY ./setup_repo.sh /root/
RUN /bin/bash /root/setup_repo.sh

WORKDIR /testbed/



In [28]:
print(test_spec.install_repo_script)

#!/bin/bash
set -euxo pipefail
git clone -o origin https://github.com/sphinx-doc/sphinx /testbed
chmod -R 777 /testbed
cd /testbed
git reset --hard 9bb204dcabe6ba0fc422bf4a45ad0c79c680d90b
git remote remove origin
source /opt/miniconda3/bin/activate
conda activate testbed
echo "Current environment: $CONDA_DEFAULT_ENV"
sed -i 's/pytest/pytest -rA/' tox.ini
sed -i 's/Jinja2>=2.3/Jinja2<3.0/' setup.py
sed -i 's/sphinxcontrib-applehelp/sphinxcontrib-applehelp<=1.0.7/' setup.py
sed -i 's/sphinxcontrib-devhelp/sphinxcontrib-devhelp<=1.0.5/' setup.py
sed -i 's/sphinxcontrib-qthelp/sphinxcontrib-qthelp<=1.0.6/' setup.py
sed -i 's/alabaster>=0.7,<0.8/alabaster>=0.7,<0.7.12/' setup.py
sed -i "s/'packaging',/'packaging', 'markupsafe<=2.0.1',/" setup.py
sed -i 's/sphinxcontrib-htmlhelp/sphinxcontrib-htmlhelp<=2.0.4/' setup.py
sed -i 's/sphinxcontrib-serializinghtml/sphinxcontrib-serializinghtml<=1.1.9/' setup.py
python -m pip install -e .[test]



In [29]:
inst

{'repo': 'sphinx-doc/sphinx',
 'instance_id': 'sphinx-doc__sphinx-7440',
 'base_commit': '9bb204dcabe6ba0fc422bf4a45ad0c79c680d90b',
 'patch': 'diff --git a/sphinx/domains/std.py b/sphinx/domains/std.py\n--- a/sphinx/domains/std.py\n+++ b/sphinx/domains/std.py\n@@ -305,7 +305,7 @@ def make_glossary_term(env: "BuildEnvironment", textnodes: Iterable[Node], index\n         term[\'ids\'].append(node_id)\n \n     std = cast(StandardDomain, env.get_domain(\'std\'))\n-    std.note_object(\'term\', termtext.lower(), node_id, location=term)\n+    std.note_object(\'term\', termtext, node_id, location=term)\n \n     # add an index entry too\n     indexnode = addnodes.index()\n@@ -565,7 +565,7 @@ class StandardDomain(Domain):\n         # links to tokens in grammar productions\n         \'token\':   TokenXRefRole(),\n         # links to terms in glossary\n-        \'term\':    XRefRole(lowercase=True, innernodeclass=nodes.inline,\n+        \'term\':    XRefRole(innernodeclass=nodes.inline,\n       